<a href="https://colab.research.google.com/github/shagufta24/Personalized_Writing_Assistant/blob/main/Train_Writing_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip install datasets accelerate faiss-cpu bitsandbytes peft transformers
!pip install numpy pandas scikit-learn torch

!pip install torch --upgrade
!pip install bitsandbytes --upgrade
!pip install transformers peft --upgrade

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import faiss
import numpy as np
import pandas as pd
import transformers
from transformers import BitsAndBytesConfig
from accelerate import infer_auto_device_map

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/My Drive/cs598PEN/"

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    print("GPU available:", torch.cuda.get_device_name(0))
else:
    print("No GPU available. Please check runtime settings.")

GPU available: Tesla T4


##Model

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Save model to drive
# path = "/content/drive/My Drive/cs598PEN/mistral-7b-v3"
# model.save_pretrained(path)
# tokenizer.save_pretrained(path)

In [ ]:
from datasets import Dataset

# Mock instruction-tuning dataset
data = {
    "instruction": ["Summarize this text:", "Translate to French:", "Generate a recipe for pancakes:"],
    "input": ["Artificial intelligence is evolving rapidly.", "The weather is sunny.", "Simple ingredients for pancakes are:"],
    "output": ["AI is advancing quickly.", "Le temps est ensoleillé.", "Flour, eggs, milk, and sugar."]
}

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

In [ ]:
# Initialize the tokenizer and add a padding token if missing
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
# Ensure the model recognizes the padding token
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
print(f"Padding token ID: {tokenizer.pad_token_id}")

Padding token ID: 32768


In [ ]:
# Define the tokenization function
def tokenize_function(example):
    # Construct the prompt and target
    prompt = f"{example['instruction']} {example['input']}\n"
    target = example["output"]

    # Tokenize input and output
    input_ids = tokenizer(prompt, truncation=True, padding="max_length", max_length=256)["input_ids"]
    labels = tokenizer(target, truncation=True, padding="max_length", max_length=256)["input_ids"]

    # Convert to PyTorch tensors
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.long)

    # Replace padding token IDs in labels with -100 (ignored in loss calculation)
    labels[labels == tokenizer.pad_token_id] = -100

    # Debugging info
    vocab_size = tokenizer.vocab_size
    print(f"Labels max value: {torch.max(labels)}, Vocabulary size: {vocab_size}")
    print(f"Input IDs shape: {input_ids.shape}, Labels shape: {labels.shape}")

    return {"input_ids": input_ids, "labels": labels}

# Tokenize the dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    remove_columns=["instruction", "input", "output"]
)

# Set dataset format for PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "labels"])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Labels max value: 29491, Vocabulary size: 32768
Input IDs shape: torch.Size([256]), Labels shape: torch.Size([256])
Labels max value: 29565, Vocabulary size: 32768
Input IDs shape: torch.Size([256]), Labels shape: torch.Size([256])
Labels max value: 29493, Vocabulary size: 32768
Input IDs shape: torch.Size([256]), Labels shape: torch.Size([256])


In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,               # Rank of the LoRA
    lora_alpha=16,     # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target attention layers
    lora_dropout=0.1,  # Dropout probability
    bias="none",       # Whether to adapt biases
    task_type="CAUSAL_LM"
)

In [ ]:
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora-finetuned-model",
    evaluation_strategy="no",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    save_steps=10,
    logging_dir="./logs",
    logging_steps=5,
    learning_rate=1e-4,
    fp16=True,  # Use mixed precision
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
class MyTrainer(Trainer):
    def training_step(self, model, inputs):
        # Add debugging prints here to inspect input data and model outputs
        # This is called for each training step, but only the first two batches of the first epoch will be printed
        if self.state.epoch < 1 and self.state.global_step < 2 * self.args.per_device_train_batch_size:
            print("Input IDs:", inputs["input_ids"])
            print("Labels:", inputs["labels"])
            print("Input IDs shape:", inputs["input_ids"].shape)
            print("Labels shape:", inputs["labels"].shape)

        # If possible, add checks to ensure labels are within the expected range (0 to vocab_size)
        # For example:
        # if torch.any(inputs["labels"] >= tokenizer.vocab_size):
        #    print("WARNING: Labels out of range")

        return super().training_step(model, inputs)


# Define Trainer
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

<ipython-input-23-ce7b96de7e82>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MyTrainer.__init__`. Use `processing_class` instead.
  trainer = MyTrainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
